# 解析初始文本文件

In [1]:
import keras
import numpy as np

Using TensorFlow backend.


In [6]:
path = r'./nietzsche.txt'

In [7]:
text = open(path).read().lower()

In [8]:
print('Corpus length:', len(text))

Corpus length: 600901


# 将字符序列向量化

In [9]:
maxlen = 60
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('Number of sequences:', len(sentences))

Number of sequences: 200281


In [10]:
chars = sorted(list(set(text)))
print('Unique characters:', len(chars))

Unique characters: 60


In [14]:
char_indices = dict((char, chars.index(char)) for char in chars)

In [18]:
print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1

Vectorization...


# 构建网络

## 用于预测下一个字符的单层LSTM模型

In [19]:
from keras import layers

In [20]:
model = keras.models.Sequential()
model.add(layers.LSTM(units=128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(units=len(chars), activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               96768     
_________________________________________________________________
dense_1 (Dense)              (None, 60)                7740      
Total params: 104,508
Trainable params: 104,508
Non-trainable params: 0
_________________________________________________________________


## 模型编译配置

In [21]:
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(optimizer=optimizer, loss='categorical_crossentropy')

## 给定模型预测，采样下一个字符的函数

In [22]:
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

## 文本生成循环

In [23]:
import random
import sys

In [24]:
for epoch in range(1, 10):
    print('epoch', epoch)
    # 将模型训练10轮
    model.fit(x, y, batch_size=128, epochs=1)

    # 随机选择一个文本种子
    start_index = random.randint(0, len(text) - maxlen - 1)
    generated_text = text[start_index:start_index + maxlen]
    print('--- Generating with seed: "' + generated_text + '"')
    
    # 尝试一系列不同的采样温度
    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ temperature:', temperature)
        sys.stdout.write(generated_text)

        # 从种子文本开始，生成400个字符
        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.
            
            # 对下一个字符进行采样
            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

epoch 1
Epoch 1/1
200281/200281 [==============================] - 379s - loss: 1.9956   
--- Generating with seed: "


chapter vii. our virtues


214. our virtues?--it is proba"
------ temperature: 0.2



chapter vii. our virtues


214. our virtues?--it is probarition of the such and the free is the self and consideration of the selfed the free of the partion of the sure of the subtions of the such the such the suble that the sensition of the subter the sensible the subjitt the sensition of the subtic and the have the surition of the such the sumption of the subtions of the self to all the forming and the sensition that is the such the subtions and the s
------ temperature: 0.5
ng and the sensition that is the such the subtions and the serition, and the sone as he not the indivent dolige and them the mansing and the "constiontions the experience of the must in the trans; and sensitt the beenness as it is the word of the and the trust of the self much of and the such the berieved and h

unpluaily fro, mi, call-taste, poactile, "condsmucted, by toughts pare. only would that, witherners. exseminity.      free aim acts when at where so. healt any reasmead engleal feeliousekncudh
preasops. "sould? is calongous
andlthenevs
eleverate ehus it is "test mankind attenshed
ible
long, a
epoch 5
Epoch 1/1
200281/200281 [==============================] - 393s - loss: 1.4820   
--- Generating with seed: "reputation as a people of depth for prussian "smartness," an"
------ temperature: 0.2
reputation as a people of depth for prussian "smartness," and sufficient, and the soul and fact the soul and still desire the soul of the soul and condition and something to be all the soul of the soul of the something to the soul and something the soul and all the artist to be all the soul and society the soul and present to be another and sense and conditions and sufficient of the feeling of the soul and the present to be are the same the strength of the
------ temperature: 0.5
 soul and the pres

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: RuntimeWarning: divide by zero encountered in log
  This is separate from the ipykernel package so we can avoid doing imports until


enturuality
------ temperature: 1.2
po mind, of their above to mither most
rendered, enturuality what it rectusevore itick , "at a though owinging cairs
an one, kac to be
lyed   :herce, caust to doman of his hight beenhtcoroous been to detrivicaluse, chaose-earedly awain, to lifection on other generaling woild
trantsw funtres alive
far when he is noerved "the miss genern herosmad "a politicindly, a human visings which opinion, vanded to fein up.

opperism we har byotter singes invelt to eppa
epoch 7
Epoch 1/1
200281/200281 [==============================] - 369s - loss: 1.4460   
--- Generating with seed: "e same conceit
that characterises astrology. astrology presu"
------ temperature: 0.2
e same conceit
that characterises astrology. astrology presulige and such a man will and experience of the experience of the distrustful and soul of the strength, and the world of the sense and such a
princially and and the soul of the sense of the sense of the sense of the sense of the sense of the